# Projeto 1 - Ciência dos Dados

Nome: Daniel Vergamini Luna  Frussa

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import emoji
import functools
import operator



In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Ana\Documents\DANIEL\Insper\SEGUNDO SEMESTRE\Ciencia de Dados\projetos\Projeto1\Entrega Final


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'roomba.xlsx'

if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo roomba.xlsx, tudo certo


In [4]:
treino = pd.read_excel(filename, sheet_name = 'Treinamento')
teste = pd.read_excel(filename, sheet_name = 'Teste')

In [5]:
#Função de limpeza que troca alguns sinais básicos por espaços 
def cleanup(text):
    #Retira as sequências de controle essa linha de código é fundamental para construção das bases de dados já que
    #se essas sequencias de controle não forem reiradas, o compilador pode acabar embaralhando os "pd.series" a partir
    #desses comandos
    text = text.replace("\\n", '').replace("\n\n", '').replace("\n", '').replace("[", '').replace("]", '')
    
    
    #Função de limpeza muito simples que troca alguns sinais básicos por espaços
    punctuation = '[!-..:?;_—•\/|‘’“”%#&༉@‧₱₊˚~$]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    text_subbed = text_subbed.replace("rt", '').replace("https", '')
    return text_subbed
    
    return text_subbed

#Função que auxilia na limpesa do material analizado, separando emojis de outros carácteres.        
def separa_emoji(lista):
    lista_split_emoji = emoji.get_emoji_regexp().split(lista)
    lista_split_whitespace = [substr.split() for substr in lista_split_emoji]
    lista_split = functools.reduce(operator.concat, lista_split_whitespace)
    return lista_split




#lista de 'stopwords_auxiliar" é uma lista de stopword muito completa que contém palavras que não influenciam a relevância
#dos tweets
stopwords_aux = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst",
             "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af",
             "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj",
             "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am",
             "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow",
             "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear",
             "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as",
             "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully",
             "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become",
             "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind",
             "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol",
             "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by",
             "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd",
             "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon",
             "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain",
             "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq",
             "cr", "cry", "cs", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd",
             "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't",
             "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down",
             "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec",
             "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", 
             "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es",
             "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone",
             "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", 
             "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo",
             "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr",
             "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get",
             "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got",
             "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has",
             "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help",
             "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself",
             "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how",
             "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i2", "i3", "i4", "i6", "i7",
             "i8", "ia", "ib", "ibid", "ic", "id", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im",
             "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index",
             "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into",
             "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its",
             "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke",
             "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last",
             "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets",
             "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking",
             "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may",
             "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill",
             "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms",
             "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay",
             "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither",
             "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non",
             "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere",
             "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often",
             "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only",
             "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our",
             "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2",
             "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd",
             "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po",
             "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably",
             "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q",
             "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily",
             "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related",
             "relatively", "research", "respectively", "resulted", "resulting", "results", "rf", "rh", 
             "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa",
             "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see",
             "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious",
             "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes",
             "she's", "should", "shouldnt", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si",
             "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl",
             "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime",
             "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify",
             "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully",
             "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken",
             "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx",
             "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then",
             "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere",
             "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll",
             "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those",
             "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti",
             "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq",
             "tr", "tried", "tries", "truly", "try", "trying", "ts", "tt", "tv", "twelve", "twenty", "twice", "two",
             "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike",
             "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully",
             "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very",
             "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants",
             "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "went",
             "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's",
             "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres",
             "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever",
             "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "wi", "widely", "will",
             "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world",
             "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn",
             "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your",
             "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz",]


#Como o compilador dá "key error" caso uma palavra na lista stopwords não esteja presente nas bases de dados do trabalho
#usei essa lista abaixo para excluir as palavras não presentes na base de dados da lista das stopwords


stopwords_lixo = ["a","0o","0s","3a","3b","3d","6b","6o","a1","a2","a3","a4","ab","above","abst","ac","accordance","accordingly",
                "across","act","added","adj","ae","af","affected","affecting","affects","afterwards","ag","against","ah",
                "ain","ain't","aj","al","allow","allows","almost","alone","along","already","although","among","amongst",
                "amoungst","amount","announce","another","anybody","anyhow","anymore","anything","anyway","anyways",
                "anywhere","ao","ap","apart","apparently","appear","appreciate","appropriate","approximately","ar","aren",
                "aren't","arise","as","aside","ask","asking","associated","au","auth","av","available","aw","awfully","ax",
                "ay","az","b","b1","b2","b3","ba","back","bd","became","become","becomes","becoming","beforehand","begin",
                "beginning","beginnings","behind","believe","below","beside","besides","between","beyond","bi","bill",
                "biol","bj","bk","bl","bn","both","bottom","bp","br","brief","briefly","bs","bt","bu","bx","c","c1","c2",
                "c3","ca","can't","cause","causes","cc","cd","ce","certain","certainly","cf","cg","ch","changes","ci","cit",
                "cj","cl","clearly","cm","c'mon","cn","co","com","con","concerning","consequently","consider","contain",
                "containing","contains","corresponding","couldn","couldnt","couldn't","cp","cq","cr","cry","cs","ct","cu",
                "currently","cv","cx","cy","cz","d","d2","da","date","dc","dd","de","definitely","describe","described",
                "despite","detail","df","didn","didn't","dj","dk","dl","doesn","doesn't","don","don't","down","downwards",
                "dp","dr","ds","dt","du","due","dx","dy","e","e2","e3","ea","each","ec","ed","edu","ee","ef","effect","eg",
                "ei","eight","eighty","ej","el","eleven","else","elsewhere","empty","en","end","ending","entirely","eo",
                "ep","eq","er","es","especially","est","et","et-al","etc","eu","ev","everybody","everyone","everything",
                "ex","exactly","except","ey","f","f2","fa","fc","few","ff","fi","fifteen","fifth","fify","fill","fire",
                "fix","fj","fl","fn","fo","followed","following","former","formerly","forth","forty","four","fr","front",
                "fs","ft","fu","further","furthermore","fy","g","ga","gave","ge","gi","give","gives","giving","gj","gl",
                "goes","gone","gotten","gr","greetings","gs","gy","h","h2","h3","hadn","hadn't","happens","hardly","hasn",
                "hasn't","haven","haven't","hed","he'd","he'll","hello","hence","here","hereafter","hereby","herein",
                "heres","here's","hereupon","herself","he's","hh","hid","himself","hither","hj","ho","hopefully","howbeit",
                "however","how's","hr","hs","http","hu","hundred","hy","i2","i4","i6","i7","i8","ia","ib","ibid","ic","id",
                "ie","ig","ignored","ih","ii","ij","il","i'll","i'm","immediate","immediately","importance","important",
                "inasmuch","inc","indeed","index","indicate","indicated","indicates","information","inner","insofar",
                "instead","interest","invention","inward","io","ip","iq","ir","isn","isn't","it'd","it'll","it's","iv",
                "i've","ix","iy","iz","j","jj","jr","js","jt","ju","k","ke","kept","kg","kj","km","known","knows","ko",
                "l","l2","la","largely","last","lately","latter","latterly","lb","lc","le","les","lest","lets","let's",
                "lf","liked","likely","line","lj","ll","ll","ln","lo","looking","los","lr","ls","lt","ltd","m","m2","ma",
                "mainly","may","mean","means","meantime","meanwhile","merely","mg","mightn","mightn't","mill","miss",
                "ml","mn","mo","moreover","most","mostly","mr","mrs","ms","mt","mu","much","mug","must","mustn","mustn't",
                "n","n2","na","namely","nay","nc","nd","ne","near","nearly","necessarily","necessary","needn","needn't",
                "neither","nevertheless","ng","ni","nine","ninety","nj","nl","nn","non","none","nonetheless","noone","nor",
                "normally","nos","noted","nothing","novel","nowhere","nr","ns","nt","ny","o","oa","ob","obtain","obtained",
                "obviously","oc","od","often","og","oi","oj","ok","okay","ol","old","om","omitted","ones","onto","oo","op",
                "oq","ord","os","ot","others","ou","ought","ours","ourselves","overall","ow","owing","ox","oz","p",
                "p1","p2","p3","page","pagecount","pages","par","part","particular","particularly",
                "pc","pd","pe","per","perhaps","pf","ph","pi","pj","pk","pl","placed","please","plus","pm","pn","poorly",
                "possible","possibly","pp","pq","pr","predominantly","present","previously","primarily","probably",
                "promptly","proud","provides","ps","pt","pu","py","q","qj","qu","que","quickly","qv","r","r2","ra","rather",
                "rc","rd","re","readily","reasonably","recent","recently","ref","refs","regarding","regardless","regards",
                "related","relatively","research","research-,articl","respectively","resulted","resulting","results","rf",
                "rh","ri","rj","rl","rm","rn","ro","rq","rr","rs","rt","ru","rv","ry","s","s2","sa","say","sc","sd","se",
                "sec","second","secondly","section","seeing","seem","seemed","seeming","self","selves","sensible",
                "seriously","seven","several","sf","shall","shan","shan't","shed","she'd","she'll","shes","she's",
                "shouldnt","shouldn't","should've","show","showed","shown","showns","shows","si","side","significant",
                "significantly","similar","similarly","sincere","six","sixty","sj","sl","sm","sn","somebody","somehow",
                "somethan","sometime","somewhere","sorry","sp","specifically","specified","specify","specifying",
                "sq","sr","ss","st","strongly","sub","substantially","successfully","such","sufficiently","suggest",
                "sup","sy","system","sz","t","t1","t2","t3","taken","taking","tb","tc","td","te","tell","ten","tends",
                "tf","th","thank","thanks","thanx","that'll","that's","that've","theirs","themselves","thence",
                "thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","there's","thereto",
                "thereupon","there've","these","they'd","they'll","they're","they've","thickv","thin","third","thorough",
                "thoroughly","thou","thoughh","thousand","three","throug","thru","thus","ti","til","tip","tj","tl","tm",
                "tn","together","took","top","toward","towards","tp","tq","tr","tried","tries","truly","ts","tt",
                "tv","twelve","twenty","tx","u","u201d","ue","ui","uj","uk","um","un","unfortunately","unless","unlike",
                "unlikely","until","unto","uo","upon","ups","ur","useful","usefully","usefulness","ut","v","va","value",
                "various","vd","ve","ve","viz","vj","vo","vol","vols","volumtype","vq","vt","vu","w","wa","wasn","wasn't",
                "wed","we'd","we'll","b","we're","weren","werent","weren't","we've","what'll","whats","what's","whence",
                "when's","whereafter","whereas","whereby","wherein","wheres","where's","whereupon","wherever",
                "whether","whim","whither","whod","whoever","whole","who'll","whom","whomever","whos","who's",
                "whose","whys","wi","widely","willing","wish","within","wo","won","wont","won't","words","world",
                "wouldn","wouldn't","www","x","x1","x2","x3","xf","xi","xj","xk","xl","xn","xo","xs","xt","xv","xx",
                "y","y2","yet","yj","yl","youd","you'd","you'll","you're","yours","yourselves","you've","yr","ys","yt",
                "z","zi","zz","ad","again","arent","before","begins","best","better","call","came","come","comes",
                "considering","course","di","different","done","during","either","em","enough","example","far","five",
                "follows","full","gets","given","go","hasnt","help","hers","hi","home","itd","itself","keep","least",
                "let","looks","might","million","move","myself","nobody","other","outside","pas","past","po","potentially",
                "presumably","quite","really","right","run","same","seems","sent","serious","since","slightly","somewhat",
                "stop","sure","theyd","throughout","try","twice","under","use","used","uses","using","usually","vs","wants",
                "wasnt","welcome","went","whatever","whenever","without","wouldnt","youre","yourself","zero"]

#Código que fornece a lista de stopwards "filtrada" 

stopwords = []
for i in stopwords_aux:
    if i in stopwords_aux and i not in stopwords_lixo:
        stopwords.append(i)
print(stopwords)



['able', 'about', 'according', 'actually', 'after', 'all', 'also', 'always', 'am', 'an', 'and', 'any', 'anyone', 'are', 'around', 'at', 'away', 'bc', 'be', 'because', 'been', 'being', 'but', 'by', 'can', 'cannot', 'cant', 'could', 'did', 'do', 'does', 'doing', 'even', 'ever', 'every', 'everywhere', 'find', 'first', 'for', 'found', 'from', 'get', 'getting', 'going', 'got', 'had', 'has', 'have', 'having', 'he', 'her', 'hes', 'him', 'his', 'how', 'i3', 'if', 'im', 'in', 'into', 'is', 'it', 'its', 'just', 'keeps', 'know', 'later', 'less', 'like', 'little', 'look', 'made', 'make', 'makes', 'many', 'maybe', 'me', 'mine', 'more', 'my', 'name', 'need', 'needs', 'never', 'new', 'next', 'no', 'not', 'now', 'of', 'off', 'oh', 'on', 'once', 'one', 'only', 'or', 'otherwise', 'our', 'out', 'over', 'own', 'put', 'ran', 'said', 'saw', 'saying', 'says', 'see', 'seen', 'she', 'should', 'so', 'some', 'someone', 'something', 'sometimes', 'soon', 'still', 'take', 'than', 'that', 'thats', 'the', 'their', 't

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI... O produto que escolhi para construção do algorítimo é o produto de limpeza "roomba", um aspirador de pó robô automatizado.O objetivo desse algorítimo é: A partir de tweets que mencionam o produto obter informações sobre a opinião dos usuarios desse produto, a fim de descobrir se o produto mostra relevância e popularidade entre os usúarios da plataforma "twitter" e orientar uma hipotética equipe de marketing a partir das informações das opiniões dos usuários coletada por esse algorítimo.
    Foram considerados relevantes: menções sobre a qualidade do produto, menções sobre problemas técnicos de usúarios que usam a plataforma, comentarios comparáitivos a produtos semelhantes e comentários que demonstram intenção de compra do produto, assim como elogíos ou críticas específicas ao produto

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [13]:
#separando o dataframe em dois, de acordo com a relevancia feita na coragem.
treino_relevante = treino.loc[treino.Relevancia == 1]
treino_irrelevante = treino.loc[treino.Relevancia == 0]

#O codigo não le o dataframe inteiro a menos que ele esteja exposto, se não ele le apenas o "resumo" que aparece no 'print'.
#Estas funções fazem com que o conteúdo inteiro seja exíbido. 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


#========================Construção das bases de dados como feito na aula 7========================================

lista_relevante = cleanup(str(treino_relevante.Treinamento))
lista_relevante = separa_emoji(lista_relevante.lower()) #Adiciona os emojis separados pela função"separa emojis na lista_Rel
serie_relevante = pd.Series(lista_relevante) 
tabela_relevantes = serie_relevante.value_counts()
tabela_relevantes = tabela_relevantes.drop(labels=stopwords) #retira as stopwords da tabela

lista_irrelevante = cleanup(str(treino_irrelevante.Treinamento))
lista_irrelevante = separa_emoji(lista_irrelevante.lower())
serie_irrelevante = pd.Series(lista_irrelevante) #Adiciona os emojis separados pela função"separa emojis na lista_Irrel
tabela_irrelevantes = serie_irrelevante.value_counts()
tabela_irrelevantes = tabela_irrelevantes.drop(labels=stopwords)# Retira as stopwords da tabela

lista_total = cleanup(str(treino.Treinamento))
lista_total = separa_emoji(lista_total.lower())# Adiciona os emojis separados pela função"separa emojis na lista_total
serie_total  = pd.Series(lista_total)
tabela_total = serie_total.value_counts()
tabela_total = tabela_total.drop(labels=stopwords) # Retira as stopwords da tabela

#==========================================================================================================
#Calculando as probabilidades da palavra ser relevante ou irrelevante dado o conjunto completo.
P_rel = len(serie_relevante)/len(serie_total)
P_irrel = len(serie_irrelevante)/len(serie_total)


print(P_rel+P_irrel)
print(len(tabela_irrelevantes))
print(len(tabela_relevantes))
print(len(tabela_total))
print(len(treino))

1.0006817794443497
1075
1027
1934
300


In [19]:
#Essa funçao calcula a probabilidade de uma dada palavra estar no grupo relevante e de estar no grupo irrelevante.
def calcula_probabilidades(palavra):

#O trecho do código abaixo previne que o cálculo da probabilidade de determinada palavra se encaixar em um determinado grupo
#(relevante ou irrelevante) atinga o valor "0". Mesmo que essa palavra não não apareça na base de dados do grupo determinado.
#Esse procedimento é conhecido como "suavização de laplace" atingimos esse resultado ao atribuir um valor de probabilidade 
#arbítrario > 0 para todas as palavras aplicadas ao algorítimo para compensar essa adicção no valor da probabilidade
#dívidimos a soma resultante pela quantidade total de palvras na "lista_total" dessa forma o resultado da divisão nunca será
# > 1.
    
    
    smoothiner = 1 #valor arbítrario
    
    if palavra in tabela_relevantes:
        freq_R = tabela_relevantes[palavra]
    else:
        freq_R = 0
        
    if palavra in tabela_irrelevantes:
        freq_I = tabela_irrelevantes[palavra]
    else:
        freq_I = 0           
            
   
 #============================Aplicação da suavizção de laplace=============================================================
    Rel_p = (freq_R + smoothiner) / (len(lista_relevante) + len(tabela_total))
    Irrel_p = (freq_I + smoothiner) / (len(lista_irrelevante) + len(tabela_total))
    return [Rel_p, Irrel_p] #  Rel_p = P(palavra|Relevantes)  ||  Irrel_p = P(palavra|Irrelevantes)


#Essa funçao usa a anterior para calcular a probabilidade de uma dada frase, por palavra, estar no grupo relevante e
#de estar no grupo irrelevante. Então as compara e retorna a relevância da frase.
def compara_probabilidades(frase):
    frase = separa_emoji(cleanup(str(frase)).lower())
    P_R = 1
    P_I = 1    
    for palavra in frase:
        P_R *= calcula_probabilidades(palavra)[0]
        P_I *= calcula_probabilidades(palavra)[1]
    if P_R >= P_I:
        return 1
    else:
        return 0
    


In [20]:
#aplicando as funções
lista_resultados = []
for i in teste['Teste']:
    lista_resultados.append(compara_probabilidades(i))

teste["classificação"] = lista_resultados
print(lista_resultados)

[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [16]:
#"comparados" é uma tabela comparativa dos valores de relevancia atribuidos manualmente e pelo classificador
comparados = pd.crosstab(teste['Relevancia'],teste['classificação'], normalize='index')

#separando os valores da tabela 
pos_true = comparados[1][1]/2
pos_false = comparados[1][0]/2
neg_true = comparados[0][0]/2
neg_false =comparados[0][1]/2

print("Verdadeiros positivos: ", round(pos_true*100,2), "%")
print("Falsos positivos: ", round(pos_false*100, 2), "%")
print("Verdadeiros negativos: ", round(neg_true*100,2), "%")
print("Falsos negativos: ", round(neg_false*100, 2), "%")
print("-"*40)

print("Acertos: ", round((pos_true+neg_true)*100,2), "%")
print("Erros: ", round((pos_false+neg_false)*100,2), "%")
print("-"*40)






Verdadeiros positivos:  33.15 %
Falsos positivos:  18.92 %
Verdadeiros negativos:  31.08 %
Falsos negativos:  16.85 %
----------------------------------------
Acertos:  64.23 %
Erros:  35.77 %
----------------------------------------


___
# Concluindo




### Comparativo qualitativo:

A partir da análise quantitativa da tabela de resultados, pode-se concluir que o classificador apresenta uma performance mínimamente satsfatória, demonstrando um gráu de precisão razoável com bastante espaço para aperfeiçoamento. No entanto, para a realização da atividade proposta esse algorítimo se mostra como uma ferramenta bem utíl, visto que a taxa de falsos tweets relevantes se mostrou baixa (18.92%"). Ou seja, utilizando essa ferramenta é bem impróvavel que o usúario se deparece com muitos tweets irrelevantes durante sua pesquisa (que é a parte mais cansativa desse processo de análise).O maior problema dessa aplicação é sua alta taxa de falsos tweets irrelevantes que se demonstram um problema grave visto o fato que o algorítimo veta informações relevantes ao usuário.
    Uma provável causa dos problemas de performance do classificador pode ter sido uma base de dados de treinamento muito pequena para os critérios adotados para o treinamento. Possivelmente com um aumento na quantidade de tweets poderiamos aumentar a taxa de palavaras relevantes  baixo demais na quantidade de twitter de treinamentos manual a fim de melhora as taxas de frequencias das palavras verdadeiramente relevantes,
    
    
### Explicação:
   
   Não se pode aumentar a base de dados de treinamento do classificador utilizando o próprio classificador, uma vez que nesse caso o critério de seleção da relevância permaneceria inalterado e portanto o que aconteceria é que atribuição de relevância
dos novos tweets estaria sujeita a uma mesma performance de acertos do que na condição inicial do classificador sem os novos tweets.
    

### Outros usos para a aplicação do classificador naive-bayes
  
   Dentro dos contextos de aplicação para a empresa hipotética, podemos citar o uso do classificador naive-bayes para medir a quantidade de opiniões positivas e negativas em relação ao produto não somente o quanto ele é comentado assim os administradores da em presa poderiam ter uma concepção melhor ainda da adesão do "roomba" entre os usúarios da plataforma twitter. 
   
    
### Sugestão de melhorias 

   Para melhorar o código, além de uma maior base de treinamento ou um critério mais claro, pode-se pensar em analisar frases com dupla-negativa ou sarcasmo. Seria necessario analisar a montagem das frases e quantidade de uso de expressões negativas por tweet no primeiro caso, e analisar. A partir dessa iteração poderiamos criar subcategorias na classificação da relevância como comentários positivos e negativos em relação ao produto levando em consideração o sarcasmo. Assim possibilitando extrair mais informações dos tweets

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

Referencia para a funçao de limpeza de sequências de controle (e outros):\
https://pt.stackoverflow.com/questions/289388/como-remover-n-de-um-string-em-python

Referencia da biblioteca utilizada para criar a função que fornece a tabela de validação:\
[pandas.crosstab](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html)

Referencia para função de separação dos Emojis:\
https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python


Referencia para criação da lista stopwords_aux:\
[Maurotoro](https://gist.github.com/sebleier/554280)

